In [385]:
import os, shutil, re
from helpers import *
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import librosa
import librosa.display
import datetime

%matplotlib inline

In [386]:
############
# Input VARS

VALID_LABELS = ["lights-on", "turn-off"]
MOVE_WINDOW = True
FNAME = "12_04"

############
# const VARS

ROOT = os.getcwd()
AUD_ROOT = os.path.join(*[ROOT, "audio", FNAME])
IMG_ROOT = os.path.join(*[ROOT, "images_scaled", FNAME])
pattern = "[0-9]{2}_[0-9]{2}"
IMG_EXT = ".png"
VERBOSITY = 1000
MOVE_WINDOW = True
WINDOW_NB = 10
WAV_DURATION = 2 # seconds
SAMPLE_RATE = 8000
WINDOW_OFFSET = 4000
WINDOW_SIZE = SAMPLE_RATE * WAV_DURATION - WINDOW_OFFSET

CHUNK = int(WINDOW_OFFSET / WINDOW_NB)
WINDOW_START = 7
WINDOW_END = 8

###########
# check dirs

print('-' * 42)
print("Current Directory:       {}".format(ROOT))
print("Current AUDIO Directory: {}".format(AUD_ROOT))
print("CUrrent IMG Directory:   {}".format(IMG_ROOT))
print('-' * 42)

# """
# Expected directory structure:
# [INSIDE ROOT DIRECTORY]
# ---- [category] voice, no_voice
# -------- [date] 07_02, 07_09, ...
# ------------ [label] down, go, ...
# ---------------- [channel] ch1, ch2, ...
# -------------------- [wave files] *.wav
# """

------------------------------------------
Current Directory:       /Users/kyy/cerebro_train
Current AUDIO Directory: /Users/kyy/cerebro_train/audio/12_04
CUrrent IMG Directory:   /Users/kyy/cerebro_train/images_scaled/12_04
------------------------------------------


In [387]:
def preprocess(samples, sample_rate, multiplier=1):
    sr = sample_rate * multiplier
    padded = np.zeros(sr)
    samples = samples[:sr]
    padded[:samples.shape[0]] = samples
    return padded

def make_dir(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [388]:
# read wavfiles and make spectrograms

def make_spectrograms(samples, sample_rate, o_channel_path, items): 
    items += 1
    imgpath = os.path.join(o_channel_path, str(items).rjust(6, '0'))
    changed = preprocess(samples, sample_rate, 1)
    S = librosa.feature.melspectrogram(changed, sr=sample_rate, n_mels=128, fmax=512)
    log_S = librosa.power_to_db(S, ref=np.max)
    fig = plt.figure(figsize=(1.28, 1.28), dpi=100, frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.axis('off')
    librosa.display.specshow(log_S)                          
    plt.savefig(imgpath)
    plt.close()
    return items

In [389]:
# captures a section of the data and moves the window by 
# chunks to produce more data

def make_windowed_spectrograms(samples, sample_rate, o_channel_path, items): 
    for i in range(WINDOW_START, WINDOW_END + 1):
        items += 1
        imgpath = os.path.join(o_channel_path, str(items).rjust(6, '0'))
        x = i * CHUNK
        window_size = WINDOW_SIZE
        changed = samples[x:window_size+x]
        changed = preprocess(changed, sample_rate, 1)
        S = librosa.feature.melspectrogram(changed, sr=sample_rate, n_mels=128, fmax=512)
        log_S = librosa.power_to_db(S, ref=np.max)
        fig = plt.figure(figsize=(1.28, 1.28), dpi=100, frameon=False)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        plt.axis('off')
        librosa.display.specshow(log_S)                          
        plt.savefig(imgpath)
        plt.close()
    return items

In [390]:
def process_files(channel_path, o_channel_path, items):
    for file in [f for f in os.listdir(channel_path) if f.endswith(".wav")]:
        wavpath = os.path.join(channel_path, file)
        sample_rate, samples = wavfile.read(wavpath)
        if MOVE_WINDOW == True:
            items = make_windowed_spectrograms(samples, sample_rate, o_channel_path, items)
        else:
            items = make_spectrograms(samples, sample_rate, o_channel_path, items)

In [391]:
def process_channels(label_path, o_label_path):
    for channel in [d for d in os.listdir(label_path) if d.startswith("ch")]:
        print("    Channel Processing: {}".format(channel))
        items = 0
        voice = False
        ch = int(channel[2:])
        channel_path = os.path.join(label_path, channel)
        o_channel_path = os.path.join(o_label_path, channel)
        make_dir(o_channel_path)
        channel_num = channel[-1]
        process_files(channel_path, o_channel_path, items)

In [392]:
def process_labels(date_path, o_date_path):
    for label in [d for d in os.listdir(date_path) if d in VALID_LABELS]:
        print("  Label Processing: {}".format(label))
        label_path = os.path.join(date_path, label)
        o_label_path = os.path.join(o_date_path, label)
        make_dir(o_label_path)
        process_channels(label_path, o_label_path)

In [393]:
def process_dates(input_dir, output_dir, overwrite=False):
    for date in [x for x in os.listdir(input_dir) if re.match(pattern, x)]:
        print("Date Processing: {}".format(date))
        date_path = os.path.join(input_dir, date)
        o_date_path = os.path.join(output_dir, date)
        make_dir(o_date_path)
        process_labels(date_path, o_date_path)

In [394]:
def process(input_dir, output_dir, overwrite=False):
    plt.ioff()
    process_dates(input_dir, output_dir, overwrite=False)
    plt.ion()

In [395]:
##############################################
# converts splitted wavfiles into spectrograms

if not os.path.isdir(IMG_ROOT):
    os.mkdir(IMG_ROOT)

dir_pairs = {
    os.path.join(AUD_ROOT, "no_voice") : os.path.join(IMG_ROOT, "no_voice")
}

In [ ]:
for input_dir in dir_pairs:
    output_dir = dir_pairs[input_dir]
    if not os.path.isdir(output_dir):
        os.mkdir(output_dir)
    timer(process, input_dir, output_dir)

Start: 2018-12-16 10:33:01.231093
Date Processing: 12_04
  Label Processing: lights-on
    Channel Processing: ch7
    Channel Processing: ch1
    Channel Processing: ch6
    Channel Processing: ch8
    Channel Processing: ch3
    Channel Processing: ch4
    Channel Processing: ch5
    Channel Processing: ch2
  Label Processing: turn-off
    Channel Processing: ch7
    Channel Processing: ch1
